In [1]:
from css import *
from qaravan.core import Z, pretty_print_sv
from qaravan.tensorQ import StatevectorSim
from qaravan.algebraQ import is_stabilized_by

In [5]:
encoding_circ = steane_encoder()

sim = StatevectorSim(encoding_circ)
sim.run(progress_bar=False)
sv = sim.get_statevector()
pretty_print_sv(sv, 2)

'0.3536+0.0000j|0000000⟩ + 0.3536+0.0000j|0001111⟩ + 0.3536+0.0000j|0110011⟩ + 0.3536+0.0000j|0111100⟩ + 0.3536+0.0000j|1010101⟩ + 0.3536+0.0000j|1011010⟩ + 0.3536+0.0000j|1100110⟩ + 0.3536+0.0000j|1101001⟩'

In [6]:
stim_circ = qaravan_to_stim(encoding_circ)
hmat = hamming_mat()
stabilizers = css_stabilizers(hmat, hmat)
for s in stabilizers:
    print(s, is_stabilized_by(stim_circ, s))

+X_X_X_X True
+_XX__XX True
+___XXXX True
+Z_Z_Z_Z True
+_ZZ__ZZ True
+___ZZZZ True


In [7]:
# flip one of the bits and see which stabilizers are violated
erred_circ = stim_circ.copy()
erred_circ.append('X', [0])
for s in stabilizers:
    print(s, is_stabilized_by(erred_circ, s))

+X_X_X_X True
+_XX__XX True
+___XXXX True
+Z_Z_Z_Z False
+_ZZ__ZZ True
+___ZZZZ True


In [8]:
# deterministic error with a decoder circuit
error = Circuit([Z(4)])
circ = encoding_circ + error + css_decoder(hmat, hmat)
stim_circ = qaravan_to_stim(circ)
stim_circ.append('M', [7+i for i in range(6)])
pretty_sample(stim_circ, num_samples=500)

101000: 500


In [9]:
# stochastic error with a decoder circuit
enc_stim = qaravan_to_stim(encoding_circ)
dec_stim = qaravan_to_stim(css_decoder(hmat, hmat))

p = 0.5
err_stim = stim.Circuit()
err_stim.append("Z_ERROR", [0], p)
full_stim = enc_stim + err_stim + dec_stim
full_stim.append("M", [7 + i for i in range(6)])
pretty_sample(full_stim, num_samples=500)

000000: 269
100000: 231


In [10]:
# simulating a cycle of the Steane code

num_sites = 7
px = 0.01
pz = 0.01

enc_stim = qaravan_to_stim(steane_encoder())
hmat = hamming_mat()

err = stim.Circuit()
err.append("Z_ERROR", [i for i in range(num_sites)], pz)
err.append("X_ERROR", [i for i in range(num_sites)], px)

full_stim = css_circuit(enc_stim, err, hmat, hmat)
logical_fails = [cycle(enc_stim, full_stim, hmat, hmat) for _ in range(10000)]
print(f"Logical failure rate: {1-np.mean(logical_fails)}")

Logical failure rate: 0.0039000000000000146
